## ✅ 템플릿 매칭 (Template Matching)

### 📌 핵심
- 작은 이미지를 큰 이미지에서 찾아내는 방법.
- 퍼즐처럼 템플릿을 이미지 위에 슬라이딩하며 비교.

### 🟢 장점
- 간단하고 빠름.
- 정확한 위치 찾기에 유리.

### 🔴 단점
- 회전, 크기 변화, 밝기 변화에 매우 약함.
- 템플릿과 거의 똑같아야 잘 작동함.

## ✅ 히스토그램 매칭 (Histogram Matching)

### 📌 핵심
- 이미지의 색상 분포(색조, 분위기)를 비교.
- 어떤 물체가 어디 있는지는 모르지만, 분위기 비슷한 이미지를 찾는 데 유용.

### 🟢 장점
- 크기나 회전 변화에 강함.
- 전체 이미지 분위기 비교에 적합.

### 🔴 단점
- 위치 정보가 없음.
- 색은 비슷하지만 모양이 전혀 다른 이미지도 비슷하다고 판단할 수 있음.

## ✅ HOG (Histogram of Oriented Gradients)

### 📌 핵심
- 이미지의 경계선 방향(gradient)을 분석해서 구조적 특징을 파악.
- 사람이나 물체의 윤곽선을 중심으로 인식.
- 딥러닝 등장 전까지 사람 검출에서 가장 많이 쓰이던 방법.

### 🟢 장점
- 밝기 변화, 회전, 크기 변화에 강함.
- 위치 정보 유지하면서 구조적 패턴 파악 가능.
- 딥러닝 없이도 꽤 좋은 성능.

### 🔴 단점
- 계산량이 많아 느릴 수 있음.
- 딥러닝 방식보다는 정확도가 낮음.

## 🧾 한눈에 비교

| 항목          | 템플릿 매칭       | 히스토그램 매칭    | HOG                        |
|---------------|--------------------|---------------------|-----------------------------|
| 목적          | 패턴 위치 찾기     | 색상 분위기 비교     | 구조 기반 물체 인식        |
| 위치 정보     | 있음               | 없음                | 있음                        |
| 회전/크기 변화 | 약함               | 강함                | 강함                        |
| 속도          | 빠름               | 빠름                | 중간~느림                   |
| 딥러닝 필요    | ❌                 | ❌                  | ❌ (전통적 방식)             |
| 대표 활용     | 로고 탐지, 아이콘  | 유사 이미지 검색     | 사람 검출, 객체 인식        |


In [1]:
# 설치 전 cMake 다운받아야함
# https://cmake.org/download/

In [2]:
%pip install dlib

  Using cached dlib-20.0.0.tar.gz (3.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for dlib: filename=dlib-20.0.0-cp38-cp38-win_amd64.whl size=2945399 sha256=227ac56784864146c0e532a0b7ef9f395ac513c2537b0323e87baeba8f19d4dd
  Stored in directory: c:\users\main\appdata\local\pip\cache\wheels\81\22\d1\c303b7964cfddb3fab9ba7aff4c7436c8c6798e9b6edb5e05f
Successfully built dlib
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
import os
from PIL import Image
import dlib

image = cv2.imread('bts.jpg')
image_resized = cv2.resize(image, (755, 500))

# HOG 검출기로 얼굴 인식
hog_face_detector = dlib.get_frontal_face_detector()
face_detections = hog_face_detector(image_resized, 1)

print(face_detections)

for face_detection in face_detections:
    left, top, right, bottom = face_detection.left(), face_detection.top(), face_detection.right(), face_detection.bottom()
    cv2.rectangle(image_resized, (left, top), (right, bottom), (0,255,0), 2)

cv2.imshow("HOG", image_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

# CNN 으로 성능개선하기
* 성능은 확실히 좋으나 시간이 매우 오래걸림

In [2]:
import cv2
import numpy as np
import os
from PIL import Image
import dlib
import time

image = cv2.imread('bts.jpg')
image_resized = cv2.resize(image, (755, 500))
image_resized_CNN = image_resized.copy()

start = time.time()

# HOG 검출기로 얼굴 인식
hog_face_detector = dlib.get_frontal_face_detector()
face_detections = hog_face_detector(image_resized, 1)

end = time.time()

print(face_detections)

for face_detection in face_detections:
    left, top, right, bottom = face_detection.left(), face_detection.top(), face_detection.right(), face_detection.bottom()
    cv2.rectangle(image_resized, (left, top), (right, bottom), (0,255,0), 2)

print(f'{end-start:.5f} HOG 검출 걸린 초')

start = time.time()

cnn_face_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')
face_detections_CNN = cnn_face_detector(image_resized_CNN,1)

end = time.time()

for idx, face_detection_CNN in enumerate(face_detections_CNN):
    left, top, right = face_detection_CNN.rect.left(), face_detection_CNN.rect.top(), face_detection_CNN.rect.right(),
    bottom, confidence = face_detection_CNN.rect.bottom(), face_detection_CNN.confidence
    print(f'confidence {idx+1}:    bottom, confidence = face_detection_CNN.rect. {confidence}')
    cv2.rectangle(image_resized_CNN, (left, top), (right, bottom), (0, 255,0),2)

print(f'{end-start:.5f} CNN 검출 걸린 초')
print(face_detections_CNN)

cv2.imshow("HOG", image_resized)
cv2.imshow("CNN", image_resized_CNN)
cv2.waitKey(0)
cv2.destroyAllWindows()

rectangles[[(446, 101) (498, 153)], [(212, 179) (287, 254)]]
0.32487 HOG 검출 걸린 초
confidence 1:    bottom, confidence = face_detection_CNN.rect. 1.118628740310669
confidence 2:    bottom, confidence = face_detection_CNN.rect. 1.1111477613449097
confidence 3:    bottom, confidence = face_detection_CNN.rect. 1.0967752933502197
confidence 4:    bottom, confidence = face_detection_CNN.rect. 1.0851155519485474
confidence 5:    bottom, confidence = face_detection_CNN.rect. 1.080199956893921
confidence 6:    bottom, confidence = face_detection_CNN.rect. 1.0618482828140259
confidence 7:    bottom, confidence = face_detection_CNN.rect. 1.0562626123428345
17.16882 CNN 검출 걸린 초


# WebCam 을 이용하여 HOG 및 CNN 테스트

In [ ]:
import cv2
import dlib
import math
import time

webcam = cv2.VideoCapture(0)
if not webcam.isOpened():
    print('Could not open webcam')
    exit()

hog_face_detector = dlib.get_frontal_face_detector()

while webcam.isOpened():
    status, frame = webcam.read()

    image_resized = cv2.resize(frame, (755, 500))

    start = time.time()
    face_detections = hog_face_detector(image_resized, 1)
    end = time.time()
    for face_detection in face_detections:
        left, top, right, bottom = face_detection.left(), face_detection.top(), face_detection.right(), face_detection.bottom()
        cv2.rectangle(image_resized, (left, top), (right, bottom), (255,0,0), 1)

    org = (50,100)
    text = f'{end - start:.5f} HOGs'
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(image_resized, text, org, font,1,(255,0,0),2)
    if status:
        cv2.imshow("test", image_resized)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
webcam.release()
cv2.destroyAllWindows()

In [3]:
import cv2
import dlib
import math
import time

webcam = cv2.VideoCapture(0)
if not webcam.isOpened():
    print('Could not open webcam')
    exit()

cnn_face_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')

while webcam.isOpened():
    status, frame = webcam.read()

    image_resized = cv2.resize(frame, (755, 500))

    start = time.time()
    face_detections = cnn_face_detector(image_resized, 1)
    end = time.time()
    for idx, face_detection_CNN in enumerate(face_detections):
        left, top, right, bottom, confidence = face_detection_CNN.rect.left(), face_detection_CNN.rect.top(), face_detection_CNN.rect.right(), face_detection_CNN.rect.bottom(), face_detection_CNN.confidence
        print(f'confidence{idx+1}: {confidence}')
        cv2.rectangle(image_resized, (left, top), (right, bottom), (255,0,0), 1)

    org = (50,100)
    text = f'{end - start:.5f} CNNs'
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(image_resized, text, org, font,1,(255,0,0),2)
    if status:
        cv2.imshow("test", image_resized)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
webcam.release()
cv2.destroyAllWindows()

confidence1: 1.1132179498672485
confidence1: 1.1143461465835571
confidence1: 1.0724029541015625
confidence1: 1.1200357675552368
